## Preparing Inputs For Model Generation

In [1]:
from datasets import load_dataset 
from transformers import AutoProcessor 

civil_plans_dict=load_dataset("genesis1SubHub/NHA-Civil-set")
civil_plans=civil_plans_dict["train"]

model_id="Qwen/Qwen2.5-VL-72B-Instruct"
processor = AutoProcessor.from_pretrained(
    model_id,
    max_pixels=12_845_056,
    use_fast=True, # default, for qwen2.5 processor
)

2025-04-19 22:34:07.787802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745102047.808266   11834 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745102047.814482   11834 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745102047.830694   11834 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745102047.830715   11834 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745102047.830717   11834 computation_placer.cc:177] computation placer alr

README.md:   0%|          | 0.00/354 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/96.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [2]:
from src.qwen_2_5_utils import setup_model

model=setup_model()

Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

## Recursive Generation

Analogous to GN building scope breakouts

### Prompts

In [7]:
testing_system_prompt = '''You're a helpful assistant project manager in the construction industry, helping put together an estimate for a multipage construction blueprint.
Here's an example of your task:
    You found these three scopes from imaginary page 'MnCFwsoI6P':
        Current Scopes:
            ```python
            [
                ("Remove existing interior walls as specified." , ["MnCFwsoI6P"]),
                ("Provide and install 7 roooftop mechanical units." , ["MnCFwsoI6P"]),
                ("Design wood frame roof truss system" , ["MnCFwsoI6P"] ),
            ]
            ```
    And on page 'DnzNkJzu8R' you saw a reference to "Design wood frame roof truss system" in addition to new scopes: 
        Current Scopes:
            ```python
            [
                ("Remove existing interior walls as specified." , ["MnCFwsoI6P"]),
                ("Provide and install 7 roooftop mechanical units." , ["MnCFwsoI6P"]),
                ("Design wood frame roof truss system" , ["MnCFwsoI6P" , 'DnzNkJzu8R'] ),
                ("Provide and apply PT1 pain to all rooms" , ['DnzNkJzu8R']),
                ("Provide and install LESCO electrical boxes as specified in the construction blueprints", ['DnzNkJzu8R']),
            ]
            ```
Your goal is to keep updating the list of current scopes to be as accurate as possible. DO NOT ADD additional observations, notes, explanations, and/or comments. Focus on being as comprehensive as possible.  

Notes:
    1. Scopes CANNOT be found on pages involving general notes, cover pages, and in the 'EXISTING' section of a legend.
    2. Some pages may not have scopes. 
    3. Features on pages CAN be scopes:
        a. On page reference: "Trash enclosure see architecturals" -> Scope: "Provide and install trash enclosure"
        b. On page reference: "Cow tongue downspout nozzle" -> Scope: "Provice and install cow tongue downspout nozzles"
    4. Scopes can be found in sections of the page with additional notes. 
'''
testing_prompt='''Using page '{page_id}', update current and add new scopes as necessary.
    Current Scopes:
        ```python
            [
{current_scopes}
        ]
        ```
'''

### Feature Extract Prompt

In [4]:
feature_system_prompt=''' You're a helpful assistant identifying relevant features to scopes of work in this set of construction blueprints.
Here's an example of your task: 
    You found these features from imaginary page 'MnCFwsoI6P':
        Current Features:
            ```python
            [
                "ASPHALT SHINGLE ROOF GAF - TIMBERLINE HOZ - COLOR BARKWOOD",
                "ROOF ACCESS PATCH",
                "ROOF GRAIIN AND OVERFLOW SEE DETAIL 5/A-105",
                "1|6BS",
                "2|6BS",
                "MOP AND BROOM HOLDER",
                "UTILITY SINK",
                "R-30 CLOSED CELL INSULATION TYPICAL ALL SLOPED ROOFS",
                "EXISTING INTERIOR WALLS TO BE DEMO",
            ]
            ```
Notes:
    1. Scopes CANNOT be found on pages involving general notes, cover pages, and in the 'EXISTING' section of a legend.
    2. Some pages may not have scopes. 
    3. DO NOT ADD additional observations, notes, explanations, and/or comments.
    4. DO NOT ADD any features with "ESMT", "EASEMENT", "easement", and/or "LOT" in them.
'''
feature_prompt="Using this page, extract all features that could be relevant to scopes of work."

### Feature -> Scope Prompt

In [5]:
convert_to_scope_system_prompt='''You're a helpful assistant in the construction industry. 
Given a list of features and the page they were extracted from convert them to scopes of work. Here's an example of your task:
    Given features from imaginary page 'MnCFwsoI6P':
        Given Features:
            ```python
            [
                "ASPHALT SHINGLE ROOF GAF - TIMBERLINE HOZ - COLOR BARKWOOD",
                "ROOF ACCESS HATCH",
                "ROOF DRAIN AND OVERFLOW SEE DETAIL 5/A-105",
                "1|6BS",
                "1X6 CEDAR BOARD FENCE PAINT - NAVAJO BEIGE (MATCH BUILDING)",
                "R-30 CLOSED CELL INSULATION TYPICAL ALL SLOPED ROOFS"
                "EXISTING INTERIOR WALLS TO BE DEMO",
            ]
            ```
        Current Scopes:
            ```python
            [
                ("Provide and install asphalt shingle roof gaf - timberline hoz - color barkwood.", ["MnCFwsoI6P"] ),
                ("Provide and install roof access hatch.", ["MnCFwsoI6P"]),
                ("Provide and install roof drain and overflow see detail 5/a-105.", ["MnCFwsoI6P"]),
                ("Provide and install 1|6BS walls.", ["MnCFwsoI6P"]),
                ("Provide and install 1x6 cedar board fence", ["MnCFwsoI6P"]),
                ("Provide and apply navajo beige paint to cedar board fence, ["MnCFwsoI6P"]),
                ("Provide and install R-30 closed cell insulation typical all sloped roofs." ["MnCFwsoI6P"]),
                ("Existing interior walls to be demo." ["MnCFwsoI6P"]),
            ]
            ```
Notes:
    1. DO NOT ADD phrases similar to "as indicated on the plan" to scopes.
    2. DO NOT ADD any scopes with "ESMT", "EASEMENT", "easement" in them.
    3. DO NOT ADD any scopes with "LOT" in them.
'''
convert_to_scope_feature_prompt="Using page '{page_id}', and these features '{additional_features_list}' convert the relevant features to scopes of work."

### Update Current Scopes Prompt

In [6]:
update_scopes_system_prompt='''You're a helpful assistant in the construction industry. 

Notes:
    1. DO NOT ADD phrases similar to "as indicated on the plan" to scopes.
    2. DO NOT ADD any scopes with "ESMT", "EASEMENT", "easement" in them.
    3. DO NOT ADD any scopes with "LOT" in them.
'''

update_scopes_prompt='''Given a list of new scopes on page '{page_id}' and current scopes combine scopes and their pages that are similar enough that they could be a single scope.
    Current Scopes:
        ```python
            [
{current_scopes}
        ]
        ```
    New Scopes: 
        ```python
            [
{new_scopes}
        ]
        ```
'''

### Utility Functions

In [7]:
import torch

In [8]:
def format_list_vertically(
    items:list[str], 
    indent_level=4
):
    indent = " " * (indent_level * 4)
    return "\n".join(
        f"{indent}{item}" for item in items
    )

In [9]:
import re
import ast
from typing import Any, Optional

# Clean up and escape inner quotes
def safe_extract_python_list(text: str):
    match = re.search(r"```python\n(\[.*?\])\n```", text, re.DOTALL)
    if not match:
        print("No list block found.")
        return None

    list_str = match.group(1)

    # Escape inner double quotes that may break string boundaries (e.g., 0'-6")
    # Fix: Replace 0'-6" → 0'-6\" inside quoted strings
    list_str = re.sub(r'(?<=\d)\"(?=[^,]*")', r'\\"', list_str)

    try:
        return ast.literal_eval(list_str)
    except Exception as e:
        print(f"Failed to parse list: {e}")
        return None

def extract_python_list_block(text: str) -> Optional[list[Any]]:
    """
    Overview:
        Extracts a Python list from a ```python ... ``` block inside a string.
    Args:
        text (str): The input string containing a code block.
    Returns:
        Optional[list[Any]]: Parsed list if found and valid, else None.
    """
    match = re.search(r"```python\n(\[.*?\])\n```", text, re.DOTALL)
    if match:
        try:
            list_str = match.group(1)
            return ast.literal_eval(list_str)
        except (SyntaxError, ValueError):
            return safe_extract_python_list(text)
    return None

In [10]:
def generate_single_input(
    processor,
    prompt:str,
    system_prompt:str,
    image,
    page_id:str,
)->dict[torch.tensor]:
    
    input_generation_start=time.time()
    width,height=get_resized_dims(image)
    if system_prompt:
        messages = [
            {
                "role" : "system" , "content" : system_prompt
            },
        ] 
    messages+=[
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image, "resized_height": height,"resized_width": width},
                {"type": "text", "text": prompt
                },
            ]
        },
    ]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    print(
        f"Shape of input_ids: {inputs['input_ids'].shape} | Input prep time elasped: {time.time() - input_generation_start:.4f}"
    )
    return inputs

In [28]:
def process_generated_ids_qwen(
    inputs:dict[torch.tensor],
    generated_ids:torch.tensor,
    processor,
):
    '''
    Returns:
        Count of new tokens generated, string version of the model outputs
    '''
    trimmed_ids=[
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    decomposed_output_text=processor.batch_decode(
        trimmed_ids,
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=False
    )
    if len(decomposed_output_text) == 1:
        return len(trimmed_ids[0]), decomposed_output_text[0]
    else:
        raise ValueError(
            f" 'decomposed_output_text' has length not equal to 1!"
        )

In [40]:
from PIL.PngImagePlugin import PngImageFile

def generate_recursive_outputs(
    prompt:str,
    system_prompt:str,
    processor,
    model,
    image:PngImageFile,
    page_id:str,
    max_new_tokens:int=1000,
)->tuple[int,str]:
    
    inputs=generate_single_input(
        processor=processor,
        prompt=prompt,
        system_prompt=system_prompt,
        image=image,
        page_id=page_id,
    )
    
    start_time=time.time()
    generated_ids_intermediate_states=model.generate(
        **inputs.to(sample_device),
        max_new_tokens=max_new_tokens,
    )
    new_token_count,decomposed_output_text=process_generated_ids_qwen(
        inputs=inputs,
        generated_ids=generated_ids_intermediate_states,
        processor=processor,
    )
    print(
        f"Generation time elapsed: {time.time() - start_time:.4f} | New tokens generated: {new_token_count}"
    )
    print(
        f"{decomposed_output_text}\n"
    )
    return new_token_count,decomposed_output_text

In [41]:
def reset_memory():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

In [42]:
def clean_llm_text(decomposed_output_text:str):
    additional_features_list=extract_python_list_block(
        decomposed_output_text
    )
    blacklist_words=(
        "esmt","easement","lot"
    )
    if additional_features_list:
        additional_features_list=list(
            set(additional_features_list)
        )
        additional_features_list = [
            feature for feature in additional_features_list if not any(
                k in feature.lower() for k in blacklist_words
            )
        ]
    return additional_features_list

### Execute

In [43]:
from src.qwen_2_5_utils import (
    process_vision_info,
    get_resized_dims,
)
from src.eval_utils import generate
from collections import defaultdict
import time

max_new_feature_tokens=1000
max_new_scope_tokens=1000
max_new_master_scope_tokens=1200

sample_device="cuda:0"
testing=False
start=3
end=5

In [44]:
current_scopes=""
current_scopes_list=[]
for idx,row in enumerate(
    (civil_plans.select(range(start, end)) if testing else civil_plans)
):
    # Civil plans header ['image', 'page_number', 'page_id']
    image=row["image"]
    page_id=row["page_id"]
    
    new_token_count,decomposed_output_text = generate_recursive_outputs(
        prompt=feature_prompt,
        system_prompt=feature_system_prompt,
        processor=processor,
        model=model,
        image=image,
        page_id=page_id,
        max_new_tokens=max_new_feature_tokens
    )
    # Returns list of strings. 
    additional_features_list=clean_llm_text(decomposed_output_text)
    
    # clear memory 
    reset_memory()

    # Convert features to scopes 
    new_token_count,decomposed_output_text = generate_recursive_outputs(
        prompt=convert_to_scope_feature_prompt.format(
            page_id=page_id,
            additional_features_list=additional_features_list
        ),
        system_prompt=convert_to_scope_system_prompt,
        processor=processor,
        model=model,
        image=image,
        page_id=page_id,
        max_new_tokens=max_new_scope_tokens,
    )

    # Returns list of tuples, doesn't make sense to set.
    additional_scopes_list=extract_python_list_block(
        decomposed_output_text
    )
    # clear memory 
    reset_memory()

    # Merge current and news scopes in a 3rd pass to the llm.
    if additional_scopes_list:
        current_scopes_list+=additional_scopes_list
    #     else:
    #         # Convert features to scopes 
    #         inputs=generate_single_input(
    #             processor=processor,
    #             prompt=update_scopes_prompt.format(
    #                 page_id=page_id,
    #                 current_scopes=current_scopes_list,
    #                 new_scopes=additional_scopes_list,
    #             ),
    #             system_prompt=update_scopes_system_prompt,
    #             image=image,
    #             page_id=page_id,
    #         )
    #         start_time=time.time()
    #         generated_ids_intermediate_states=model.generate(
    #             **inputs.to(sample_device),
    #             max_new_tokens=max_new_master_scope_tokens,
    #         )
    #         new_token_count,decomposed_output_text=process_generated_ids_qwen(
    #             inputs=inputs,
    #             generated_ids=generated_ids_intermediate_states,
    #             processor=processor,
    #         )
    #         print(
    #             f"Generation time elapsed: {time.time() - start_time:.4f} | New tokens generated: {new_token_count}"
    #         )
    #         print(
    #             f"{decomposed_output_text}\n"
    #         )
    #         # clear memory 
    #         reset_memory()
    
# took 2071 seconds to execute 

Shape of input_ids: torch.Size([1, 11181]) | Input prep time elasped: 1.1181
Generation time elapsed: 9.9948 | New tokens generated: 7
```python
[
]
```

Shape of input_ids: torch.Size([1, 11384]) | Input prep time elasped: 0.7572
Generation time elapsed: 25.1069 | New tokens generated: 82
Since the provided features list is empty (`[]`), there are no specific features to convert into scopes of work for the given page `b09CcDg5Ib`. Therefore, no scopes of work can be generated based on the current input.

If you have additional features or details that should be included, please provide them, and I can assist in converting those into appropriate scopes of work.

Shape of input_ids: torch.Size([1, 11181]) | Input prep time elasped: 0.8284
Generation time elapsed: 9.9043 | New tokens generated: 7
```python
[
]
```

Shape of input_ids: torch.Size([1, 11384]) | Input prep time elasped: 0.6226
Generation time elapsed: 25.1199 | New tokens generated: 82
Since the provided features list is em

In [47]:
scopes=[i[0] for i in current_scopes_list]

len(scopes),len(set(scopes))

(366, 354)

In [48]:
from collections import defaultdict

results_dict=defaultdict(list)
for scope, pages in current_scopes_list:
    results_dict["Scopes"].append(scope)
    results_dict["Page"].append(pages)

In [49]:
import pandas as pd 

df=pd.DataFrame(results_dict)
df.to_csv("NHA_civil_recursive_generation_results2.csv",index=False)

In [50]:
df

,Scopes,Page
0,Provide and install a building as per the desi...,[q07wbgRPl3]
1,Construct a ditch/swale as per the provided de...,[q07wbgRPl3]
2,Install water hydrant and valve as per the spe...,[q07wbgRPl3]
3,Install sanitary and manhole as per the design...,[q07wbgRPl3]
4,Install water manhole as per the provided deta...,[q07wbgRPl3]
...,...,...
361,Provide and install roof grain and overflow se...,[BCr4RwQomw]
362,Provide and install R-30 closed cell insulatio...,[BCr4RwQomw]
363,Provide and install asphalt shingle roof gaf -...,[BCr4RwQomw]
364,Provide and install mop and broom holder.,[BCr4RwQomw]
